In [1]:
import pandas as pd
import numpy as np

In [12]:
import json
with open("./Accord_all_contract_NER_lower.json", "r") as f:
    templates = json.load(f)

In [13]:
templates.keys()

dict_keys(['helloworld', 'signature-block-title-name-date', 'latedeliveryandpenalty-currency-conversion', 'payment-upon-delivery', 'supply-agreement-loc', 'latedeliveryandpenalty', 'promissory-note', 'latedeliveryandpenalty-optional-this', 'rental-deposit-with', 'certificate-of-incorporation', 'empty', 'car-rental-tr', 'fragile-goods', 'supplyagreement', 'promissory-note-md', 'lateinvoicewithpayment', 'bill-of-lading', 'volumediscount', 'perishable-goods', 'servicelevelagreement', 'supplyagreement-perishable-goods', 'docusign-po-failure', 'fixed-interests-static', 'company-information', 'volumediscountolist', 'contact-information', 'acceptance-of-delivery', 'installment-sale', 'hellomodule', 'copyright-license', 'saft', 'minilatedeliveryandpenalty-capped', 'payment-upon-iot', 'volumediscountulist', 'latedeliveryandpenalty-optional', 'safte', 'interest-rate-swap', 'docusign-connect', 'demandforecast', 'rental-deposit', 'sales-contract-ru', 'empty-contract', 'one-time-payment-tr', 'simpl

In [14]:
results = {}

import tqdm
from transformers import AutoTokenizer
# b) Load model & tokenizer
model_name = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

for name, template in tqdm.tqdm(templates.items()):


    import difflib

    d = difflib.Differ()


    predict_y = []
    for i, token in enumerate(template['NER']['prediction']):
        try:
            if token['prediction'][0]['type'] == "O":
                predict_y.append(0)
            else:
                predict_y.append(1)
        except:
            if token['predictions'][0]['type'] == "O":
                predict_y.append(0)
            else:
                predict_y.append(1)


    sample = tokenizer.tokenize("".join(template['/text/sample.md']))
    grammar = tokenizer.tokenize("".join(template['/text/grammar.tem.md']))

    diff = d.compare(sample, grammar)
    diff = list(diff)

    sample_with_mark = list(filter(lambda x: x[0] in ["-", " "], diff))

    true_y = []
    for token in sample_with_mark:
        if token[0] == "-" and token not in ['- Ġ"', '- "', '".'] and '"' not in token and "'" not in token:
            true_y.append(1)
        else:
            true_y.append(0)

    np.array(true_y) == np.array(predict_y)
    results[name] = {"true_y": np.array(true_y), "predict_y": np.array(predict_y)}

  0%|                                   | 0/56 [00:00<?, ?it/s]/home/eason/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
Token indices sequence length is longer than the specified maximum sequence length for this model (3306 > 512). Running this sequence through the model will result in indexing errors
100%|█████████████████████████| 56/56 [00:00<00:00, 116.17it/s]


In [15]:
len(results)

56

In [23]:
result['true_y'] == result['predict_y']

/home/eason/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [21]:
result['predict_y']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
(result['true_y'] == result['predict_y'])

/home/eason/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [16]:
total = np.array([])
total_true_y = np.array([])
total_pred_y = np.array([])
for name, result in results.items():
    total = np.append(total, [(result['true_y'] == result['predict_y']).astype(int)])
    total_true_y = np.append(total_true_y, [(result['true_y']).astype(int)])
    total_pred_y = np.append(total_pred_y, [(result['predict_y']).astype(int)])

/home/eason/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


AttributeError: 'bool' object has no attribute 'astype'

In [ ]:
total.mean()

In [ ]:
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y_actual[i]==y_pred[i]==1:
            TP += 1
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
            FP += 1
        if y_actual[i]==y_pred[i]==0:
            TN += 1
        if y_pred[i]==0 and y_actual[i]!=y_pred[i]:
            FN += 1

    return(TP, FP, TN, FN)

In [ ]:
TP, FP, TN, FN = perf_measure(total_true_y, total_pred_y)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
confusion_matrix(total_true_y, total_pred_y, labels=[0, 1])

In [73]:
f1_score(total_true_y, total_pred_y, labels=[0, 1])

0.311721332316298

In [67]:
TP

1839

In [47]:
FP

1

In [48]:
TN

8134

In [49]:
FN

0

In [29]:
np.append([1, 2, 3], [[4, 5, 6], [7, 8, 9]])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0])

[{'type': 'O', 'confidence': 0.9404129981994629},
 {'type': 'B-org', 'confidence': 0.24135534465312958},
 {'type': 'Organization', 'confidence': 0.2282179296016693}]

In [98]:
name

'helloworld'

In [100]:
predict_y

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0]

In [99]:
true_y

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]

In [28]:
"".join(template['/text/grammar.tem.md'])

'Upon the signing of this Agreement, {{buyer}} shall pay {{amount}} to {{seller}}.'

In [29]:
"".join(template['/text/sample.md'])

'Upon the signing of this Agreement, "Dave" shall pay 50 USD to "Dan".'

In [82]:
ans = pd.DataFrame({"true_y": true_y, "predict_y": predict_y})

In [83]:
ans

,true_y,predict_y
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,1
6,0,0
7,0,0
8,1,1
9,0,0


In [69]:
sample_with_mark

['  Upon',
 '  Ġthe',
 '  Ġsigning',
 '  Ġof',
 '  Ġthis',
 '  ĠAgreement',
 '  ,',
 '- Ġ"',
 '- Dave',
 '- "',
 '  Ġshall',
 '  Ġpay',
 '- Ġ50',
 '- ĠUSD',
 '  Ġto',
 '- Ġ"',
 '- Dan',
 '- ".']

['  Upon',
 '  Ġthe',
 '  Ġsigning',
 '  Ġof',
 '  Ġthis',
 '  ĠAgreement',
 '  ,',
 '- Ġ"',
 '- Dave',
 '- "',
 '+ Ġ{{',
 '+ buy',
 '+ er',
 '+ }}',
 '  Ġshall',
 '  Ġpay',
 '- Ġ50',
 '- ĠUSD',
 '+ Ġ{{',
 '+ amount',
 '+ }}',
 '  Ġto',
 '- Ġ"',
 '- Dan',
 '- ".',
 '+ Ġ{{',
 '+ seller',
 '+ }}',
 '+ .']

2023-02-09 11:49:04.071346: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [24]:
list(diff)

['  Upon',
 '  Ġthe',
 '  Ġsigning',
 '  Ġof',
 '  Ġthis',
 '  ĠAgreement',
 '  ,',
 '- Ġ"',
 '- Dave',
 '- "',
 '+ Ġ{{',
 '+ buy',
 '+ er',
 '+ }}',
 '  Ġshall',
 '  Ġpay',
 '- Ġ50',
 '- ĠUSD',
 '+ Ġ{{',
 '+ amount',
 '+ }}',
 '  Ġto',
 '- Ġ"',
 '- Dan',
 '- ".',
 '+ Ġ{{',
 '+ seller',
 '+ }}',
 '+ .']

In [12]:
difference("".join(template['/text/sample.md']), "".join(template['/text/grammar.tem.md']))


The Difference Between Two Strings: 
{'50', '{{seller}}.', '{{buyer}}', '"Dave"', 'USD', '"Dan".', '{{amount}}'}
The programs runs successfully.


In [10]:
def difference(string1, string2):
      # Split both strings into list items
  string1 = string1.split()
  string2 = string2.split()

  A = set(string1) # Store all string1 list items in set A
  B = set(string2) # Store all string2 list items in set B
 
  str_diff = A.symmetric_difference(B)
  isEmpty = (len(str_diff) == 0)
 
  if isEmpty:
    print("No Difference. Both Strings Are Same")
  else:
    print("The Difference Between Two Strings: ")
    print(str_diff)
  
  print('The programs runs successfully.')

# Driver code to call a function
usr_str1 = 'Educative is good'
usr_str2 = 'Educative is bad'
output = difference(usr_str1, usr_str2)


The Difference Between Two Strings: 
{'good', 'bad'}
The programs runs successfully.


In [7]:
template

{'/request.json': {'$class': 'org.accordproject.payment.PaymentReceived'},
 '/.cucumber.js': ["require('@accordproject/cicero-test').Steps;"],
 '/logic/logic.ergo': ['/*\n',
  ' * Licensed under the Apache License, Version 2.0 (the "License");\n',
  ' * you may not use this file except in compliance with the License.\n',
  ' * You may obtain a copy of the License at\n',
  ' *\n',
  ' * http://www.apache.org/licenses/LICENSE-2.0\n',
  ' *\n',
  ' * Unless required by applicable law or agreed to in writing, software\n',
  ' * distributed under the License is distributed on an "AS IS" BASIS,\n',
  ' * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n',
  ' * See the License for the specific language governing permissions and\n',
  ' * limitations under the License.\n',
  ' */\n',
  '\n',
  'namespace org.accordproject.payment.uponsignature\n',
  '\n',
  'import org.accordproject.runtime.*\n',
  'import org.accordproject.obligation.*\n',
  'import org.accordproject